In [10]:
import numpy as np

In [1]:
# drake imports
from pydrake.all import (
    DiagramBuilder,
    MeshcatVisualizer,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    StartMeshcat,
)
from pydrake.examples import PendulumGeometry, PendulumPlant

In [2]:
from underactuated import running_as_notebook
from underactuated.meshcat_utils import MeshcatSliders

In [3]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [6]:
def pendulum_simulation():
    builder = DiagramBuilder()
    pendulum = builder.AddSystem(PendulumPlant())

    # Visualization setup
    scene_graph = builder.AddSystem(SceneGraph())
    PendulumGeometry.AddToBuilder(builder, pendulum.get_state_output_port(), scene_graph)

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    meshcat.Delete()

    meshcat.Set2dRenderMode(
        X_WC=RigidTransform(RotationMatrix.MakeZRotation(np.pi), [0,1,0])
    )

    # set slider input
    meshcat.AddSlider("u", min=-5, max=5, step=0.1, value = 0.0)
    torque_system = builder.AddSystem(MeshcatSliders(meshcat, ["u"]))
    builder.Connect(torque_system.get_output_port(), pendulum.get_input_port())

    diagram = builder.Build()

    # set up simulator

    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    meshcat.AddButton("Stop Simulation")

    # set initial conditions
    context.SetContinuousState([0.5,0]) # theta - angular pos., theta_dot - ang. vel.

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)

        print("Use slider to apply torque")
        print("Press stop simulation to stop")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)
    else:
        simulator.AdvanceTo(0.1)

    meshcat.DeleteAddedControls()
    

    
    

In [5]:
pendulum_simulation()

Use slider to apply torque
Press stop simulation to stop


In [9]:
RotationMatrix.MakeZRotation(np.pi)

RotationMatrix([
  [-1.0, -1.2246467991473532e-16, 0.0],
  [1.2246467991473532e-16, -1.0, 0.0],
  [0.0, 0.0, 1.0],
])